In [ ]:
pip install -q --upgrade langchain langchain-openai langchain-core langchain_community langchain_chroma sentence_transformers docx2txt pypdf

In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Try loading .env
load_dotenv()

# Try Colab secrets if .env doesn't have it
try:
    from google.colab import userdata
    open_ai_key = os.getenv("OPENAI_API_KEY") or userdata.get("OPENAI_API_KEY")
    langchain_api_key = os.getenv("LANGCHAIN_API_KEY") or userdata.get("LANGCHAIN_API_KEY")
except:
    open_ai_key = os.getenv("OPENAI_API_KEY")
    langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

print(open_ai_key)
print(langchain_api_key)

In [ ]:
os.environ["OPENAI_API_KEY"] = open_ai_key

os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = 'langchain-course'
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tracers.context import tracing_v2_enabled

llm = ChatOpenAI(model_name="gpt-4o-mini")
query_to_llm = input("Enter your query: ")
while len(query_to_llm) == 0:
  print('Query cannot be empty. Please try again.\n')
  query_to_llm = input("Enter your query: ").strip()

with tracing_v2_enabled():
  llm_resonse = llm.invoke(query_to_llm)

llm_resonse

In [ ]:
# parsing output
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_resonse)

In [ ]:
# Simple Chain
chain = llm | output_parser
chain.invoke(query_to_llm)

In [ ]:
# Structured Output

from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
  phone_model: str = Field(description="Name and Model of the phone")
  rating: int = Field(description="Rating of the phone out of 5")
  pros: List[str] = Field(description="List of positive aspects")
  cons: List[str] = Field(description="List of negative aspects")
  summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S24 Ultra and I can't believe. The screen is stunning colors pop like crazy.
The camera is insane especially AI mode that captures Moon, the night mode and the anti shake videography.
Battery life is solid, an all day long with surfing, gaming, consuming media.

Not goona lie though, its pretty pricey. And what's with ditching the charger? C'mon Samsung don't try to be cheap like Apple.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say its a solid 4.5 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you are due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)

output

In [ ]:
output.pros

In [ ]:
output.cons

In [ ]:
# Prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
prompt.invoke({"topic": "programming"})


In [ ]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "programmer"})